In [1]:
print("hello")

hello


In [10]:
# Basics to Kuzu Graph databases
# Our example: 4 Nodes, where data is stored in a dataframe


import pandas as pd

id_number = [1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009]

departments = ['hr', 'hr', 'it', 'media', 'design', 'design', 'dev', 'dev', 'dev', 'dev' ]

colors = ['red', 'green', 'yellow', 'orange', 'red', 'green', 'yellow', 'orange', 'yellow', 'red']

role = ['intern', 'junior', 'senior', 'manager', 'senior', 'manager', 'junior', 'senior', 'senior', 'manager']



dict = {'id_number':id_number, 'departments':departments, 'colors':colors, "role":role}



#-----------------------------------

our_data = pd.DataFrame(dict)

our_data

,id_number,departments,colors,role
0,1000,hr,red,intern
1,1001,hr,green,junior
2,1002,it,yellow,senior
3,1003,media,orange,manager
4,1004,design,red,senior
5,1005,design,green,manager
6,1006,dev,yellow,junior
7,1007,dev,orange,senior
8,1008,dev,yellow,senior
9,1009,dev,red,manager


In [4]:
# Every directory needs database path (db) to store the Kuzu Database
# I also suggest a directory that can hold your notebooks and datasets

In [1]:
import kuzu
import os


# Start by initializing your Kuzu database
# and create a connection for executing queries
db_path = os.path.join(".", "db", "graph_db")
if not os.path.exists(db_path):
    os.makedirs(db_path)

db = kuzu.Database(db_path)
conn = kuzu.Connection(db)



In [2]:
# Next, create a node table
    # Each Node must have a 
# Follow Kuzu's syntax


# Use conn.execute to run a Kuzu query
# CREATE NODE TABLE is a command to create a node
# NAME_OF_THE_NODE( PROPERTY and DATA TYPE, PRIMARY KEY(PROPERTY) )


conn.execute(""" 
    CREATE NODE TABLE Employee(
        id INT64,
        PRIMARY KEY(id)
    )
""")
conn.execute("""
    CREATE NODE TABLE Department(
        name STRING,
        PRIMARY KEY(name)
    )
""")
conn.execute("""
    CREATE NODE TABLE Color(
        name STRING,
        PRIMARY KEY(name)
    )
""")
conn.execute("""
    CREATE NODE TABLE Role(
        name STRING,
        PRIMARY KEY(name)
    )
""")



# Every Node table created must have at least one property. One property is the primary key
# This ensures each Node is unique



In [4]:
# Now  define relationship tables. 

# Intuitively, 'Employee' connects to every node (FROM Employee TO others)
conn.execute("CREATE REL TABLE WORKS_IN(FROM Employee TO Department)")
conn.execute("CREATE REL TABLE LEAVING_COLOR(FROM Employee TO Color)")
conn.execute("CREATE REL TABLE HAS_ROLE(FROM Employee TO Role)")



# There may be additional relationships that you want to model
# For example:
    # Departments collaborate with each other (e.g. it helps dev)
conn.execute("CREATE REL TABLE COLLABORATES_WITH(FROM Department TO Department)")

    # Colors indicate turnover risk causing hiring needs
conn.execute("CREATE REL TABLE CAUSES_HIRING_NEED(FROM Color TO Department)")

    # Roles are linked to specific departments (e.g., manager role doesn't exist only in hr and it).
conn.execute("CREATE REL TABLE REQUIRES_ROLE(FROM Department TO Role)")


RuntimeError: Connection is closed.